In [1]:
#x01_populationStatistics

In [2]:
import matplotlib.pyplot as plt
import uuid
import pdb
import torch
from matplotlib.animation import FuncAnimation
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import gymnasium as gym
import numpy as np
device='cpu'

### BREEDING SIMULATOR
class Genome:
    def __init__(self, n_chr, n_loci):
        self.ploidy = 2
        self.n_chr = n_chr
        self.n_loci = n_loci
        self.shape = (self.ploidy, self.n_chr, self.n_loci)

class Population:
    def __init__(self, genome, haplotypes, device=device):
        self.genome = genome
        self.device = device
        self.phenotypes = None
        self.bvs = None
        self.haplotypes = haplotypes.to(device)
        self.dosages = haplotypes.sum(dim=1).float().to(device)
        self.size = haplotypes.shape[0]

class Trait:
    def __init__(self, genome, founder_population, target_mean, target_variance, device=device):
        self.target_mean = target_mean
        self.target_variance = target_variance
        self.device = device
        random_effects = torch.randn(genome.n_chr, genome.n_loci, device=self.device)
        random_effects -= random_effects.mean()
        founder_scores = torch.einsum('kl,hkl->h', random_effects, founder_population.dosages).to(device)
        founder_mean, founder_var = founder_scores.mean(), founder_scores.var()
        scaling_factors = torch.sqrt(self.target_variance / founder_var)
        self.scaling_factors = scaling_factors
        random_effects *= scaling_factors
        self.effects = random_effects
        self.intercept = founder_mean - target_mean


def calculate_breeding_value(population, trait, device=device):
    return torch.einsum('hjk,jk->h', population.dosages, trait.effects).to(device)

def truncation_selection(population, trait, top_percent):
    return torch.topk(population.phenotypes, top_percent).indices.to(device)

# meiosis
def recombine(parent_haplo_tensor, recombination_rate=0.1):
    num_individuals, ploidy, num_chromosomes, num_loci = parent_haplo_tensor.shape
    # Generate crossover masks
    maternal, paternal = parent_haplo_tensor[:,0,:,:], parent_haplo_tensor[:,1,:,:]
    crossovers = torch.bernoulli(torch.full((num_individuals, num_chromosomes, num_loci), recombination_rate, device=device))
    progeny = maternal * (1 - crossovers) + paternal * crossovers
    return progeny

def phenotype(population, trait, h2):
    breeding_values = calculate_breeding_value(population, trait)
    population.breeding_values = breeding_values
    if breeding_values.var() == 0:
        environmental_variance = 0
    else:
        environmental_variance = (1 - h2) / h2 * breeding_values.var()

    # Check if environmental_variance is zero before applying torch.sqrt and .clone()
    if environmental_variance == 0:
        environmental_noise = torch.zeros(breeding_values.shape, device=device)
    else:
        environmental_noise = torch.randn(breeding_values.shape, device=device) * torch.sqrt(environmental_variance).detach()

    population.breeding_values = breeding_values
    population.phenotypes = breeding_values + environmental_noise

    return population.phenotypes.max()

def create_random_pop(G, pop_size):
    return torch.randint(0, 2, (pop_size, *G.shape), device=device)

def truncation_selection(population, trait, top_percent):
    return torch.topk(population.phenotypes, top_percent).indices.to(device)

# meiosis
def recombine(parent_haplo_tensor, recombination_rate=0.1):
    num_individuals, ploidy, num_chromosomes, num_loci = parent_haplo_tensor.shape
    # Generate crossover masks
    maternal, paternal = parent_haplo_tensor[:,0,:,:], parent_haplo_tensor[:,1,:,:]
    crossovers = torch.bernoulli(torch.full((num_individuals, num_chromosomes, num_loci), recombination_rate, device=device))
    progeny = maternal * (1 - crossovers) + paternal * crossovers
    return progeny

def phenotype(population, trait, h2):
    breeding_values = calculate_breeding_value(population, trait)
    population.breeding_values = breeding_values
    if breeding_values.var() == 0:
        environmental_variance = 0
    else:
        environmental_variance = (1 - h2) / h2 * breeding_values.var()

    # Check if environmental_variance is zero before applying torch.sqrt and .clone()
    if environmental_variance == 0:
        environmental_noise = torch.zeros(breeding_values.shape, device=device)
    else:
        environmental_noise = torch.randn(breeding_values.shape, device=device) * torch.sqrt(environmental_variance).detach()

    population.breeding_values = breeding_values
    population.phenotypes = breeding_values + environmental_noise

    return population.phenotypes.max()

def create_random_pop(G, pop_size):
    return torch.randint(0, 2, (pop_size, *G.shape), device=device)

def update_pop(population, haplotype_pop_tensor):
    population.haplotypes = haplotype_pop_tensor
    population.dosages = haplotype_pop_tensor.sum(dim=1).float()
    return population

# meiosis
def recombine(parent_haplo_tensor, recombination_rate=0.1):
    num_individuals, ploidy, num_chromosomes, num_loci = parent_haplo_tensor.shape
    # Generate crossover masks
    maternal, paternal = parent_haplo_tensor[:,0,:,:],parent_haplo_tensor[:,1,:,:],
    crossovers = torch.bernoulli(torch.full((num_individuals, num_chromosomes, num_loci), recombination_rate, device=device))
#     crossovers = torch.rand((num_individuals, num_chromosomes, num_loci), device=device) < recombination_rate
    progeny = maternal * torch.logical_not(crossovers) + paternal * crossovers
    return progeny

def breed(mother_tensor, father_tensor, recombination_rate=0.1):
    eggs = recombine(mother_tensor,recombination_rate)
    pollens = recombine(father_tensor,recombination_rate)
    return torch.stack((eggs,pollens), dim=1)

def create_pop(G, haplotypes):
    return Population(G, haplotypes=haplotypes)

def bv(P,T):
    P.breeding_values = calculate_breeding_value(P.dosages,T.effects)

def create_progeny(mother_gametes, father_gametes, reps=1, device=device):
    progeny = []
    for _ in range(reps):
        # Randomly shuffle the gametes from each parent
        shuffled_mother_indices = torch.randperm(mother_gametes.shape[0], device=device)
        shuffled_father_indices = torch.randperm(father_gametes.shape[0], device=device)

        # Select the shuffled gametes
        mother_gametes = mother_gametes[shuffled_mother_indices]
        father_gametes = father_gametes[shuffled_father_indices]

        # Stack the gametes to create progeny haplotypes
        progeny_haplotypes = torch.stack((mother_gametes, father_gametes), dim=1)
        progeny.append(progeny_haplotypes)
    return torch.vstack(progeny)

class SimParams:
    def __init__(self,G,T,h2,reps,pop_size,max_generations,founder_pop):
        self.G = G
        self.T = T
        self.h2 = h2
        self.reps = reps
        self.pop_size = pop_size
        self.max_generations = max_generations
        self.founder_pop = founder_pop

#RL ENVIRONMENT
class BreedingEnvironment(gym.Env):
    def __init__(self, SP):
        super(BreedingEnvironment, self).__init__()
        self.SP = SP
        self.current_generation = 0
        self.max_generations = SP.max_generations


        # Define action and observation space
        self.action_space = gym.spaces.Discrete(5)  # 0, 1, 2, 3, or 4
        self.action_history = []

        self.observation_space = gym.spaces.Dict({
            "population": gym.spaces.Box(low=0, high=1, shape=(200, 2, 1, 200), dtype=np.int32),
            "generation": gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        })

        self._action_to_direction = {0:(200,1),
               1:(100,2),
               2:(50,4),
               3:(25,8),
               4:(5,40),}

    def _get_obs(self):
        return {
            "population": self.population.haplotypes.cpu(),
            "generation": torch.tensor([self.generation / self.SP.max_generations], dtype=torch.float32).cpu()
        }

    def _get_info(self):
        return {
            "phenotype": self.phenotype.cpu().item(),
            "genetic_variance": self.population.breeding_values.var().cpu().item()
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.population = self.SP.founder_pop
        self.phenotype = phenotype(self.population, self.SP.T, self.SP.h2)
        self.generation = 0

        observation = self._get_obs()
        info = self._get_info()
        self.action_history = []
        return observation, info

    def step(self, action):
        # Map the action (element of {0,1,2,3}) to the direction we walk in
#         print(action)
        total_parents, total_crosses = self._action_to_direction[int(action)] # top parents, number crosses per

        top_k = torch.topk(self.population.phenotypes, total_parents).indices
        selected = self.population.haplotypes[top_k]

        # Breeding
        m = recombine(selected)  # Mother gametes
        f = recombine(selected)  # Father gametes
        progeny = create_progeny(m, f, reps=total_crosses)

        # Create new population from progeny
        self.population = create_pop(self.SP.G, progeny)
        self.phenotype = phenotype(self.population, self.SP.T, self.SP.h2)


        self.generation+=1
        observation = self._get_obs()
        info = self._get_info()
        terminated = self.generation >= self.SP.max_generations
        truncated = False
        reward = self.phenotype
        self.action_history.append((action))
        self.current_generation += 1
        done = self.current_generation >= self.max_generations

        if done:
            info['final_generation'] = {
                'phenotype': self.phenotype,
                'genetic_variance': self.population.breeding_values.var().item()
            }
        return observation, reward, terminated, truncated, info


#RLAGENT

import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define our custom feature extractor
class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 64):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim)

        population_shape = observation_space['population'].shape
        self.input_size = population_shape[0] * population_shape[1] * population_shape[3]  # Flattened population size

        self.fc1 = nn.Linear(self.input_size + 1, features_dim).to(device)  # +1 for generation input
        self.fc2 = nn.Linear(features_dim, features_dim).to(device)

    def forward(self, observations):
        population = observations['population'].float().to(device)
        generation = observations['generation'].float().to(device)

        # Flatten the population input
        x = population.view(population.size(0), -1)

        # Concatenate generation input
        x = torch.cat([x, generation], dim=1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

# Define our custom policy
class CustomActorCriticPolicy(ActorCriticPolicy):
    def __init__(self, observation_space, action_space, lr_schedule, *args, **kwargs):
        super(CustomActorCriticPolicy, self).__init__(
            observation_space,
            action_space,
            lr_schedule,
            net_arch=[dict(pi=[64, 64], vf=[64, 64])],
            features_extractor_class=CustomFeatureExtractor,
            features_extractor_kwargs=dict(features_dim=8),
            *args,
            **kwargs
        )

#LOGGING
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np
import torch
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback

import numpy as np
import torch
from stable_baselines3.common.callbacks import BaseCallback

import numpy as np
import torch
from stable_baselines3.common.callbacks import BaseCallback

class AverageFinalGenerationCallback(BaseCallback):
    def __init__(self, log_freq=100, verbose=0):
        super(AverageFinalGenerationCallback, self).__init__(verbose)
        self.log_freq = log_freq
        self.phenotypes = []
        self.genetic_variances = []
        self.best_phenotype = -float('inf')
        self.episode_count = 0

    def _on_step(self) -> bool:
        for env_idx, done in enumerate(self.locals['dones']):
            if done:
                info = self.locals['infos'][env_idx]
                if 'final_generation' in info:
                    self.episode_count += 1
                    final_gen_info = info['final_generation']
                    
                    # Move tensor to CPU and convert to numpy
                    phenotype = final_gen_info['phenotype']
                    if isinstance(phenotype, torch.Tensor):
                        phenotype = phenotype.cpu().numpy()
                    self.phenotypes.append(phenotype)
                    
                    genetic_variance = final_gen_info['genetic_variance']
                    if isinstance(genetic_variance, torch.Tensor):
                        genetic_variance = genetic_variance.cpu().numpy()
                    self.genetic_variances.append(genetic_variance)

                    # Update best phenotype
                    self.best_phenotype = max(self.best_phenotype, phenotype)

                    # Log every log_freq episodes
                    if self.episode_count % self.log_freq == 0:
                        avg_phenotype = np.mean(self.phenotypes)
                        avg_genetic_variance = np.mean(self.genetic_variances)

                        self.logger.record("final_generation/avg_phenotype", avg_phenotype)
                        self.logger.record("final_generation/avg_genetic_variance", avg_genetic_variance)
                        self.logger.record("final_generation/best_phenotype", self.best_phenotype)

                        # Reset lists for next logging period
                        self.phenotypes = []
                        self.genetic_variances = []

        return True


class BaselineTensorboardCallback(BaseCallback):
    def __init__(self, baselines, log_freq=1000, verbose=0):
        super(BaselineTensorboardCallback, self).__init__(verbose)
        self.baselines = baselines
        self.log_freq = log_freq

    def _on_step(self) -> bool:
        if self.n_calls % self.log_freq == 0:
            for action, value in self.baselines.items():
                self.logger.record(f"baselines/action_{action}", value)
            
            # Log the best baseline for comparison
            best_baseline = max(self.baselines.values())
            self.logger.record("baselines/best", best_baseline)

        return True

n_chr = 1
n_loci = 200
founder_pop_size = 200
h2 = 1
reps=1
max_generations=5
G = Genome(n_chr, n_loci)
founder_pop = create_pop(G, create_random_pop(G, founder_pop_size))
T = Trait(G, founder_pop, target_mean=0.0, target_variance=1.0)
SP = SimParams(G,T,h2, reps, founder_pop_size, max_generations, founder_pop)

from stable_baselines3 import PPO
from gymnasium.spaces import Box
import gymnasium.spaces as spaces

#BASELINES
# Create the environment
env = BreedingEnvironment(SP)
env.reset()
#calculate naive (e.g. choosing the same option every time)
results = {}
REPS=30
for a in range(5):
    env.reset()
    results[a] = []
    for r in range(REPS): #reps
        env.reset()
        for i in range(env.SP.max_generations):
            env.step(a)
        results[a].append(env.phenotype.cpu().numpy())
random_results = []
baselines = {}
for i in results.keys():
    baselines[i] = (np.array(results[i]).mean())

print(baselines)

{0: 2.7271245, 1: 4.927556, 2: 5.9933896, 3: 6.1237473, 4: 5.015103}


In [3]:


tensorboard_log = "./ppotb"


# for i in [2,5,10]:
#   #hyperparameters
#   bs = i
#   lr = .0003
#   n_steps = 10
#   total_timesteps=250000
#   log_freq = 1

#   from stable_baselines3.common.callbacks import CallbackList

#   # Create the BaselineTensorboardCallback
#   baseline_callback = BaselineTensorboardCallback(baselines, log_freq=log_freq)  # Adjust log_freq as needed

#   # Create the AverageFinalGenerationCallback
#   avg_gen_callback = AverageFinalGenerationCallback(log_freq=log_freq, verbose=0)

#   # Combine both callbacks into a CallbackList
#   callback_list = CallbackList([baseline_callback, avg_gen_callback])

#   # Create the model
#   model = PPO(CustomActorCriticPolicy, env, batch_size=bs, n_steps=n_steps, learning_rate=lr, device='cuda', verbose=0,
#               tensorboard_log=tensorboard_log)

#   # Set the tensorboard log name
#   tb_log_name = f"PPO_bs_{bs}_nsteps_{bs*10}"

#   # Train the model with both callbacks
#   model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name, callback=callback_list)

In [4]:
class GenerationStatsCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(GenerationStatsCallback, self).__init__(verbose)
        self.generation_phenotypes = []
        self.generation_genetic_variances = []
        self.max_generations = None
        self.episode_count = 0

    def _on_step(self) -> bool:
        env = self.training_env.envs[0]
        
        if self.max_generations is None:
            self.max_generations = env.SP.max_generations
            self.generation_phenotypes = [[] for _ in range(self.max_generations)]
            self.generation_genetic_variances = [[] for _ in range(self.max_generations)]

        current_gen = env.generation
        phenotype = env.phenotype.cpu().item()
        genetic_variance = env.population.breeding_values.var().cpu().item()

        self.generation_phenotypes[current_gen].append(phenotype)
        self.generation_genetic_variances[current_gen].append(genetic_variance)

        if env.generation >= env.SP.max_generations - 1:
            self.episode_count += 1
            if self.episode_count % 100 == 0:  # Log every 100 episodes
                self._log_stats()

        return True

    def _log_stats(self):
        for gen in range(self.max_generations):
            avg_phenotype = np.mean(self.generation_phenotypes[gen])
            avg_genetic_variance = np.mean(self.generation_genetic_variances[gen])
            
            self.logger.record(f"generation_stats/avg_phenotype_gen_{gen}", avg_phenotype)
            self.logger.record(f"generation_stats/avg_genetic_variance_gen_{gen}", avg_genetic_variance)

        # Reset the lists for the next logging period
        self.generation_phenotypes = [[] for _ in range(self.max_generations)]
        self.generation_genetic_variances = [[] for _ in range(self.max_generations)]

In [5]:


tensorboard_log = "./ppotb"


bs = i
lr = .00003
n_steps = 10
total_timesteps=25000
log_freq = 1

from stable_baselines3.common.callbacks import CallbackList

# Create the BaselineTensorboardCallback
baseline_callback = BaselineTensorboardCallback(baselines, log_freq=log_freq)  # Adjust log_freq as needed

# Create the AverageFinalGenerationCallback
avg_gen_callback = AverageFinalGenerationCallback(log_freq=log_freq, verbose=0)
generation_stats_callback = GenerationStatsCallback()

# Combine both callbacks into a CallbackList
callback_list = CallbackList([baseline_callback, avg_gen_callback, generation_stats_callback])

# Create the model
model = PPO(CustomActorCriticPolicy, env, batch_size=bs, n_steps=n_steps, learning_rate=lr, device='cuda', verbose=0,
          tensorboard_log=tensorboard_log)

# Set the tensorboard log name
tb_log_name = f"abcPPO_bs_{bs}_nsteps_{bs*10}"

# Train the model with both callbacks
model.learn(total_timesteps=total_timesteps, tb_log_name=tb_log_name, callback=callback_list)

/home/glect/miniconda3/envs/breeding/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 4, but because the `RolloutBuffer` is of size `n_steps * n_envs = 10`, after every 2 untruncated mini-batches, there will be a truncated mini-batch of size 2
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=10 and n_envs=1)
  warnings.warn(
/home/glect/miniconda3/envs/breeding/lib/python3.11/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(
/home/glect/miniconda3/envs/breeding/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.SP to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `

KeyboardInterrupt: 

In [ ]:
from stable_baselines3.common.callbacks import CallbackList

# Create the callbacks
gen_stats_callback = GenerationStatsCallback()
avg_final_gen_callback = AverageFinalGenerationCallback(log_freq=10)
baseline_callback = BaselineTensorboardCallback(baselines, log_freq=10)

# Combine the callbacks
callback_list = CallbackList([gen_stats_callback, avg_final_gen_callback, baseline_callback])
# Set the tensorboard log name
tb_log_name = f"testgen_lowerLr"

# Use the callback list when training your model
model = PPO(CustomActorCriticPolicy, env, verbose=1, tensorboard_log=tensorboard_log, learning_rate=.0008)
model.learn(total_timesteps=1000000,tb_log_name=tb_log_name, callback=callback_list)
